In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"
import argparse
import time
import csv
import sys
import json
import random
import numpy as np
import pprint
import yaml
from torch import nn

import torch
import torch.multiprocessing as mp

import ray
from ray import tune

from matdeeplearn import models, process, training

config_path = 'config.yml'
#os.path.exists(config_path)
# os
os.path.abspath(os.getcwd())

assert os.path.exists(config_path), (
    "Config file not found in " + config_path
  )
with open(config_path, "r") as ymlfile:
    config = yaml.load(ymlfile, Loader=yaml.FullLoader)


In [2]:
config["Job"]['Evidential']

{'job_name': 'my_evidential_job',
 'reprocess': 'False',
 'model': 'MEGNet_EV_demo',
 'load_model': 'False',
 'save_model': 'True',
 'model_path': 'my_model.pth',
 'write_output': 'True',
 'parallel': 'False',
 'seed': 0}

In [3]:
config["Job"] = config["Job"]['Evidential']
config["Models"] = config["Models"].get(config["Job"]['model'])
world_size = torch.cuda.device_count()
print(world_size)
config["Processing"]["data_path"] = "data/pt_data/pt_data_2"

1


In [4]:
##General imports
import csv
import os
import time
from datetime import datetime
import shutil
import copy
import numpy as np
from functools import partial
import platform
import random
import pandas as pd

##Torch imports
import torch.nn.functional as F
import torch
from torch_geometric.data import DataLoader, Dataset
from torch_geometric.nn import DataParallel
import torch_geometric.transforms as T
from torch.utils.data.distributed import DistributedSampler
from torch.nn.parallel import DistributedDataParallel
import torch.distributed as dist
import torch.multiprocessing as mp

##Matdeeplearn imports
from matdeeplearn import models
import matdeeplearn.process as process
import matdeeplearn.training as training
from matdeeplearn.models.utils import model_summary

In [5]:
config["Models"]['coeff'] = 0.01
#config["Models"]['gc_count'] = 7
#config["Models"]['dropout_rate'] = 0.5
config["Training"]['loss'] = 'evidential'
rank = 'cuda'

In [6]:
rank = 'cuda'
print(world_size)
data_path = config["Processing"]["data_path"]
job_parameters= config["Job"]
training_parameters= config["Training"]
model_parameters= config["Models"]
processing_args= config['Processing']



1


In [7]:
"""dataset = process.get_dataset(
                config["Processing"]["data_path"],
                config["Training"]["target_index"],
                False,
                config["Processing"],
            )

dataset[0]
"""

'dataset = process.get_dataset(\n                config["Processing"]["data_path"],\n                config["Training"]["target_index"],\n                False,\n                config["Processing"],\n            )\n\ndataset[0]\n'

In [8]:
"""training.train_regular('cuda',
                       1,
                       config["Processing"]["data_path"],
                       config["Job"],
                       config["Training"],
                       model_parameters=config["Models"])"""

'training.train_regular(\'cuda\',\n                       1,\n                       config["Processing"]["data_path"],\n                       config["Job"],\n                       config["Training"],\n                       model_parameters=config["Models"])'

In [9]:
dataset = process.get_dataset(
                config["Processing"]["data_path"],
                config["Training"]["target_index"],
                config["Job"]["reprocess"],
                config["Processing"],
            )

print('Done Processing')

if rank not in ("cpu", "cuda"):
    dist.barrier()

##Set up loader
(
    train_loader,
    val_loader,
    test_loader,
    train_sampler,
    train_dataset,
    val_dataset,
    test_dataset,
) = training.loader_setup(
    training_parameters["train_ratio"],
    training_parameters["val_ratio"],
    training_parameters["test_ratio"],
    model_parameters["batch_size"],
    dataset,
    rank,
    job_parameters["seed"],
    world_size,
)



Done Processing
train length: 14850 val length: 1980 test length: 2970 unused length: 1 seed : 0


/home/varivoda/.local/lib/python3.7/site-packages/torch_geometric/deprecation.py:13: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


# CHECKING DATA TO GET FEATURES & OTHER ELEMENTS FOR BAYESIAN 

In [18]:
dir(dataset)

['__add__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_data_list',
 '_download',
 '_indices',
 '_process',
 'collate',
 'copy',
 'data',
 'data_path',
 'download',
 'get',
 'index_select',
 'indices',
 'len',
 'num_classes',
 'num_edge_features',
 'num_features',
 'num_node_features',
 'pre_filter',
 'pre_transform',
 'process',
 'processed_dir',
 'processed_file_names',
 'processed_path',
 'processed_paths',
 'raw_dir',
 'raw_file_names',
 'raw_paths',
 'root',
 'shuffle',
 'slices',
 'transform']

In [ ]:
nfeat = dataset.num_features
nclass = 1
nfeat_list = [nfeat, 128, 128, 128, 1]

#Double check that this is looking at Node and Edge Features
num_edges = dataset.num_edge_features
num_nodes = dataset.num_node_features

nlay = 4 #NUMBER OF BBGDC LAYERS

In [26]:
dataset.num_features

114

# FINISHED

In [10]:
model =training.model_setup(
    rank,
    model_parameters["model"],
    model_parameters,
    dataset,
    job_parameters["load_model"],
    job_parameters["model_path"],
    model_parameters.get("print_model", True),
)

##Set-up optimizer & scheduler
optimizer = getattr(torch.optim, model_parameters["optimizer"])(
        model.parameters(),
        lr=model_parameters["lr"],
        **model_parameters["optimizer_args"]
    )
scheduler = getattr(torch.optim.lr_scheduler, model_parameters["scheduler"])(
    optimizer, **model_parameters["scheduler_args"]
)



--------------------------------------------------------------------------
               Layer.Parameter    Param Tensor Shape              Param #
--------------------------------------------------------------------------
         pre_lin_list.0.weight            [100, 114]                11400
           pre_lin_list.0.bias                 [100]                  100
       e_embed_list.0.0.weight             [100, 50]                 5000
         e_embed_list.0.0.bias                 [100]                  100
       e_embed_list.0.2.weight            [100, 100]                10000
         e_embed_list.0.2.bias                 [100]                  100
       e_embed_list.1.0.weight            [100, 100]                10000
         e_embed_list.1.0.bias                 [100]                  100
       e_embed_list.1.2.weight            [100, 100]                10000
         e_embed_list.1.2.bias                 [100]                  100
       e_embed_list.2.0.weight      

In [ ]:
#model, optimizer, loader, loss_method, rank, 
"""loss_method = training_parameters['loss']
my_coeff = 0.2
model.train()
loss_all = 0
count = 0
for data in train_loader:
    data = data.to(rank)
    optimizer.zero_grad()
    output = model(data)
    # print(data.y.shape, output.shape)
    if loss_method == "evidential":
        loss =  training.evidential_regresssion_loss(data.y, output, 0.6)
        loss.backward()
        loss_all += loss.detach() * torch.flatten(output[0]).size(0)
        optimizer.step()
        count = count + torch.flatten(output[0]).size(0)
model(data)"""

In [ ]:


##Start training
model = training.trainer(
    rank,
    world_size,
    model,
    optimizer,
    scheduler,
    training_parameters["loss"],
    train_loader,
    val_loader,
    train_sampler,
    model_parameters["epochs"],
    training_parameters["verbosity"],
    "my_model_temp.pth",
    config["Models"]['coeff']
)



In [ ]:
train_error, train_out = training.ev_evaluate(
                train_loader, model, training_parameters["loss"], rank, True, config["Models"]['coeff']
            )
val_error, val_out = training.ev_evaluate(
                    val_loader, model, training_parameters["loss"], rank, True, config["Models"]['coeff']
                )
test_error, test_out = training.ev_evaluate(
                    test_loader, model, training_parameters["loss"], rank, True, config["Models"]['coeff']
                )
test_out

In [ ]:
train_out.to_csv(os.path.join(os.getcwd(),'Deep_Evidential_Train.csv'), index = False, header=True)
val_out.to_csv(os.path.join(os.getcwd(),'Deep_Evidential_Val.csv'), index = False, header=True)
test_out.to_csv(os.path.join(os.getcwd(),'Deep_Evidential_Test.csv'), index = False, header=True)


In [7]:
run_mode = "Evidential"
if run_mode == "Evidential":
    print("Starting Evidential training")
    print(
        "running for "
        + str(config["Models"]["epochs"])
        + " epochs"
        + " on "
        + str(config["Job"]["model"])
        + " model" 
        + " with a coefficient of "
        + str(config["Models"]['coeff'])
    )
    world_size = torch.cuda.device_count()
    if world_size == 0:
        print("Running on CPU - this will be slow")
        training.train_regular(
            "cpu",
            world_size,
            config["Processing"]["data_path"],
            config["Job"],
            config["Training"],
            config["Models"],
            config["Processing"]
        )

    elif world_size > 0:
        print("Running on one GPU")
        training.train_regular(
            "cuda",
            world_size,
            config["Processing"]["data_path"],
            config["Job"],
            config["Training"],
            config["Models"],
            config["Processing"]
        )

Starting Evidential training
running for 250 epochs on MEGNet_EV_demo model with a coefficient of 0.05
Running on one GPU
train length: 15840 val length: 990 test length: 2970 unused length: 1 seed : 0


/home/varivoda/.local/lib/python3.7/site-packages/torch_geometric/deprecation.py:13: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


--------------------------------------------------------------------------
               Layer.Parameter    Param Tensor Shape              Param #
--------------------------------------------------------------------------
         pre_lin_list.0.weight            [100, 114]                11400
           pre_lin_list.0.bias                 [100]                  100
       e_embed_list.0.0.weight             [100, 50]                 5000
         e_embed_list.0.0.bias                 [100]                  100
       e_embed_list.0.2.weight            [100, 100]                10000
         e_embed_list.0.2.bias                 [100]                  100
       e_embed_list.1.0.weight            [100, 100]                10000
         e_embed_list.1.0.bias                 [100]                  100
       e_embed_list.1.2.weight            [100, 100]                10000
         e_embed_list.1.2.bias                 [100]                  100
       e_embed_list.2.0.weight      

Epoch: 0005, Learning Rate: 0.000500, Training Error: 2.40808, Val Error: 3.02998, Time per epoch (s): 41.57778
Epoch: 0010, Learning Rate: 0.000500, Training Error: 1.95916, Val Error: 3.17225, Time per epoch (s): 40.92333
Epoch: 0015, Learning Rate: 0.000500, Training Error: 1.54533, Val Error: 2.44119, Time per epoch (s): 41.36906
Epoch: 0020, Learning Rate: 0.000500, Training Error: 1.82450, Val Error: 1.90651, Time per epoch (s): 41.66836
Epoch: 0025, Learning Rate: 0.000500, Training Error: 1.10314, Val Error: 3.88880, Time per epoch (s): 41.10982
Epoch: 0030, Learning Rate: 0.000500, Training Error: 1.07856, Val Error: 1.78495, Time per epoch (s): 41.29811
Epoch: 0035, Learning Rate: 0.000500, Training Error: 1.33673, Val Error: 2.17045, Time per epoch (s): 41.54671
Epoch: 0040, Learning Rate: 0.000500, Training Error: 1.03478, Val Error: 0.88001, Time per epoch (s): 41.06848
Epoch: 0045, Learning Rate: 0.000500, Training Error: 1.11187, Val Error: 1.31451, Time per epoch (s): 4